## Bloque I
Descomprimir el csv en formato gz

In [27]:
"""  
Importación de ficheros de AIRBNB. Preprocesado y cargado a la base de datos 

Obtenemos los datos desde https://insideairbnb.com/get-the-data/

Podríamos hacer scraping, obtener la fecha de la última actualización de Madrid y generar el enlace para descargarlo, pero no es necesario por que se actualiza cada mucho tiempo y no es necesario tener la última versión. Podemos cambiar la info procesable en la carpeta data manualmente.

"""
import os
import pandas as pd
import numpy as np
import datetime
from config_bd import *
import requests
import sys
import gzip
import shutil

DATA_DIR = os.path.join('data')
# DATA_DIR = os.path.join(os.path.dirname(__file__), 'data')
TEMP_DIR = os.path.join(DATA_DIR, 'TEMP')


""" Comprobamos que existe el directorio de datos """

if not os.path.exists(DATA_DIR):
    print(f"El directorio {DATA_DIR} no existe. No es posible continuar...")
    sys.exit()

""" Descomprimimos el archivo de calendario """

if not os.path.exists(TEMP_DIR):
    os.makedirs(TEMP_DIR)


gz_file_path = os.path.join(DATA_DIR, 'calendar.csv.gz')
output_file_path = os.path.join(TEMP_DIR, os.path.splitext(os.path.basename(gz_file_path))[0])

if os.path.exists(gz_file_path):
    try:
        with gzip.open(gz_file_path, 'rb') as f_in:
            with open(output_file_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Archivo {gz_file_path} descomprimido como {output_file_path}")
    except Exception as e:
        print(f"Error al descomprimir el archivo {gz_file_path}: {e}")
        sys.exit()
else:
    print(f"El archivo {gz_file_path} no existe. No es posible continuar...")
    sys.exit()


Archivo data\calendar.csv.gz descomprimido como data\TEMP\calendar.csv


## Bloque II
Carga del csv en pandas. Cambios de tipos e imputación

Estos son lo pasos seguidos:

<ul>
<li>Elimino el simbolo de la moneda y lo convierto a float.</li>
<li>Convierto la disponibilidad de string a bool (0,1)</li>
<li>Convierto las max y min nigths a enteros. Imputo los faltantas con las medianas de cada una.</li>
<li>Elimino el precio ajustado.</li>
<li>Renombro el listing_id a property_id</li>
</ul>

In [28]:
df = pd.read_csv(output_file_path, low_memory=False)

In [29]:
df['price'] = df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df['price'] = df['price'].fillna(0)
df['available'] = df['available'].replace({'t': 1, 'f': 0})
df['minimum_nights'] = df['minimum_nights'].fillna(df['minimum_nights'].median()).astype(int)
df['maximum_nights'] = df['maximum_nights'].fillna(df['maximum_nights'].median()).astype(int)
df = df.drop(columns=['adjusted_price'])
df = df.rename(columns={'listing_id': 'property_id'})
# df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d')

C:\Users\dsantamaria\AppData\Local\Temp\ipykernel_16060\1973130110.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['available'] = df['available'].replace({'t': 1, 'f': 0})


In [38]:
df

,property_id,date,available,price,minimum_nights,maximum_nights
730,30320,2024-09-11,0,65.0,5,180
1024,30320,2024-09-12,0,65.0,5,180
1025,30320,2024-09-13,0,65.0,5,180
1026,30320,2024-09-14,0,65.0,5,180
1027,30320,2024-09-15,0,65.0,5,180
...,...,...,...,...,...,...
9846612,1242719768352447224,2025-09-06,0,515.0,31,330
9846613,1242719768352447224,2025-09-07,0,515.0,31,330
9846614,1242719768352447224,2025-09-08,0,515.0,31,330
9846615,1242719768352447224,2025-09-09,0,515.0,31,330


Cargamos el fichero de listings ya transformado para conocer qué propiedades debemos descartar del calendar y por lo tanto no las llevamos a la base de datos.

In [30]:
listings = pd.read_csv(os.path.join(TEMP_DIR,'transform_listings.csv'))
properties = set(listings['property_id'].unique())
print(f"El número de propiedades en fichero de listings transformado es: {len(properties)}")

El número de propiedades en fichero de listings transformado es: 16805


In [31]:
print(f"El número de propiedades en el calendar es: {len(df['property_id'].unique())}, con un total de {df['property_id'].count()} registros")

El número de propiedades en el calendar es: 26980, con un total de 9847608 registros


In [32]:
df = df[df["property_id"].isin(properties)].copy()

In [33]:
print(f"El número de propiedades en el calendar filtrado es: {len(df['property_id'].unique())}, con un total de {df['property_id'].count()} registros")

El número de propiedades en el calendar filtrado es: 13842, con un total de 5052330 registros


In [34]:
# group = df.groupby(['property_id','available']).agg({'date':'count'}).reset_index().sort_values(by=['property_id','available'], ascending=[True,True]).to_csv(os.path.join(TEMP_DIR,'calendar_grouped.csv'), index=False)   

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5052330 entries, 730 to 9846616
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   property_id     int64  
 1   date            object 
 2   available       int64  
 3   price           float64
 4   minimum_nights  int64  
 5   maximum_nights  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 269.8+ MB


## Bloque III
Importamos a la base de datos. 
**Requiere haber creado la tabla, dejo el script que he utilizado para su creación en la ruta** <code>sql_querys/02_CREATE TABLE bnb_calendar.sql</code>

In [36]:
from config_bd import get_connection

In [37]:
df.to_csv(os.path.join(TEMP_DIR,'transform_calendar.csv'), index=False)

🌡️ **La carga a la base de datos del servidor por bloques tarda de 5 a 10 minutos** 🌡️

*Ya está cargado en la base de datos, sino hacemos cambios no sería necesario volverlo a correr. (después de filtrar quedan +6.000.000 registros)*.

Si se resube hacer un truncate a la table ya que tiene PK en [property_id, date], no se manejan conflictos en este proceso por lo que se rompería, si bien he hecho un groupby y no se dan dates duplicadas para el mismo property_id

In [ ]:
from tqdm import tqdm  #con esta lib itero sobre el dataframe y muestro el progreso.

with get_connection() as conn:
    cursor = conn.cursor()
    cursor.fast_executemany = True 

    CHUNKSIZE = 100000
    for chunk in tqdm(
        pd.read_csv(os.path.join(TEMP_DIR, 'transform_calendar.csv'), chunksize=CHUNKSIZE),
        unit_scale=CHUNKSIZE, unit=" filas"
    ):

        insert_sql = """
        INSERT INTO dbo.bnb_calendar
            (property_id, [date], available, price,
            minimum_nights, maximum_nights)
        VALUES (?,?,?,?,?,?)
        """
        cursor.executemany(insert_sql, chunk.values.tolist())
        conn.commit()        


from tqdm import tqdm  #con esta lib itero sobre el dataframe y muestro el progreso.

with get_connection() as conn:
    cursor = conn.cursor()
    cursor.fast_executemany = True 

    CHUNKSIZE = 100000
    for chunk in tqdm(
        pd.read_csv(os.path.join(TEMP_DIR, 'transform_calendar.csv'), chunksize=CHUNKSIZE),
        unit_scale=CHUNKSIZE, unit=" filas"
    ):

        insert_sql = """
        INSERT INTO dbo.bnb_calendar
            (property_id, [date], available, price,
            minimum_nights, maximum_nights)
        VALUES (?,?,?,?,?,?)
        """
        cursor.executemany(insert_sql, chunk.values.tolist())
        conn.commit()        




Elimino el fichero para no dejar duplicado el csv

In [ ]:
os.remove(os.path.join(TEMP_DIR,'transform_calendar.csv'))